In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
# from pinecone import Pinecone

c:\a\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
PINECONE_API_KEY = "de52f7a8-84a7-4dc0-bf34-a98394867b09"
PINECONE_API_ENV = "gcp-starter"

In [5]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("data/")


In [7]:
#CREATE TEXT CHUNKS
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)


    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
len(text_chunks)

7020

In [9]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings
embeddings = download_hugging_face_embeddings()

c:\a\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [10]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [11]:
text_chunks[0].page_content

'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION'

In [12]:
from langchain.vectorstores import Chroma
persist_directory="db"

vectordb=Chroma.from_documents(documents=text_chunks,embedding=embeddings,persist_directory=persist_directory)


In [13]:
vectordb.persist()

c:\a\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [14]:
vectordb=None
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embeddings)

In [15]:
vectordb

In [17]:
retrieval=vectordb.as_retriever()
docs=retrieval.get_relevant_documents("What are Allergies")

c:\a\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [20]:
print(docs[0].page_content)

GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE


In [22]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [23]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [24]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [27]:
# relevant_documents = retrieval.get_relevant_documents(query, search_kwargs={'k': 2})
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retrieval,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [28]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\a\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  Acneverts Acneuary Acne.
Acneeds Definally face, I don'Acnevulcervulcers Managing Skinform of course offered information, acidosis not all three types of course, Acneeds with a href= There are for moderate Persistent adults, There are lotions."  The definition
The term "Acneer.”American Academy of information you don'Acneck on. I cannot be honestly Skinformed with regard as for example
You'
Acne
Acnevulcercleanse Management

Acneeds Definally Face, Acne
Acne
There are lotions,  Yes,Acneers Managing Skinform of acneers Battle acneer’
Acne Acne Acne.
Acneverts.
Acneuary skin disorder to treatments battle their purpose --------Acne Face and kidne.
Acneering Adults Acne
The information on the user250
Acnevicts andrenalys Get informed response.
Acneers Definan experienced dermatologists Managing Skinformed, 
Acneeds Definally Face masking a doctor is that would not an importantly Obtains. Acneuary skin disease.”
Acne Acne
The best acneuestion of these three common
Hi there are m

KeyboardInterrupt: 